In [55]:
'''
2019/02/22 inoue hiromichi
最後の悪あがきです。
sklearnのSVCについて、optunaでチューニングしましたが、オーバーフィットしてしまいます。
train : 100%
test : 65%
ちなみに、optunaを使わないと、50%にも満たない精度でした(45%程度)

反省：
解析する前にデータを可視化して、データ構造をなんとなくでも理解してから、前処理をしないと精度が出なそう。
'''

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import optuna

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

def objective(y_train, y_test, X_train_std, X_test_std, trial):
    #目的関数
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear','rbf','sigmoid']),
        'C': trial.suggest_loguniform('C', 1e+1, 10.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 3.0),
    }
    mdl = SVC(**params)
    mdl.fit(X_train_std, y_train)
    pred_test = mdl.predict(X_test_std)
    accuracy_test = accuracy_score(y_test, pred_test)
    return 1.0 - accuracy_test

def main():
    global target_column
    df_original = pd.read_csv("/Users/pc1013/Desktop/first/optuna_xgboost/dataframe/winequality-red.csv")
    df = preprocess(df_original)
    
    X = df.drop(target_column,axis=1)
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.1, random_state = 666)
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    #optunaの前処理
    obj_f = partial(objective, y_train, y_test, X_train_std, X_test_std)
    #セッション作成
    study = optuna.create_study()
    #回数
    study.optimize(obj_f, n_trials=100)

    params = {}
    params['kernel'] = study.best_params['kernel']
    params['C'] = study.best_params['C']
    params['gamma'] = study.best_params['gamma']
    
    mdl = SVC(**params)
    mdl.fit(X_train_std, y_train)
    
    pred_train = mdl.predict(X_train_std)
    accuracy_train = accuracy_score(y_train, pred_train)
    pred_test = mdl.predict(X_test_std)
    accuracy_test = accuracy_score(y_test, pred_test)
    print(params)    
    print('train score :',accuracy_train)
    print('test score :',accuracy_test)

if __name__ == '__main__':
    target_column = 'quality'
    main()


[I 2019-02-22 17:35:40,689] Finished a trial resulted in value: 0.42500000000000004. Current best value is 0.42500000000000004 with parameters: {'kernel': 'linear', 'C': 10.000000000000002, 'gamma': 0.0016062003055529077}.
[I 2019-02-22 17:35:40,919] Finished a trial resulted in value: 0.42500000000000004. Current best value is 0.42500000000000004 with parameters: {'kernel': 'linear', 'C': 10.000000000000002, 'gamma': 0.0016062003055529077}.
[I 2019-02-22 17:35:40,988] Finished a trial resulted in value: 0.56875. Current best value is 0.42500000000000004 with parameters: {'kernel': 'linear', 'C': 10.000000000000002, 'gamma': 0.0016062003055529077}.
[I 2019-02-22 17:35:41,039] Finished a trial resulted in value: 0.53125. Current best value is 0.42500000000000004 with parameters: {'kernel': 'linear', 'C': 10.000000000000002, 'gamma': 0.0016062003055529077}.
[I 2019-02-22 17:35:41,118] Finished a trial resulted in value: 0.41874999999999996. Current best value is 0.41874999999999996 with 

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:41,849] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:41,871] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:41,899] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:41,925] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

...

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:42,299] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:42,341] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:42,388] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

ValueError: The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).
[W 2019-02-22 17:35:42,433] Setting trial status as TrialState.FAIL because of the following error: ValueError("The 'low' should be lower than the 'high'. But (low, high) = (2.302585092994046, 2.302585092994046).",)
Traceback (most recent call last):
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/study.py", line 400, in _run_trial
    result = func(trial)
  File "<ipython-input-55-314ea03cd02c>", line 24, in objective
    'C': trial.suggest_loguniform('C', 1e+1, 10.0),
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 181, in suggest_loguniform
    return self._suggest(name, distributions.LogUniformDistribution(low=low, high=high))
  File "/Users/pc1013/.pyenv/versions/3.6.6/lib/python3.6/site-packages/optuna/trial.py", line 394, in _suggest
    self.storage, self.study_id, name, distribution)
  Fil

{'kernel': 'rbf', 'C': 10.000000000000002, 'gamma': 0.14058601223665507}
train score : 0.8450312717164697
test score : 0.6125
